In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Reading in our file
raw_data = pd.read_csv('IMDBDataset.csv')

In [ ]:
# Get data & labels
reviews = raw_data.review
labels = raw_data.sentiment

In [ ]:
# Replace 'positive' with 1; 'negative' with 0
labels.replace({'positive': 1, 'negative':0}, inplace=True)

In [ ]:
labels

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [ ]:
patterns = ['<br />', '--', '.', ',', '!', '?', ')', '(', ';', ':', '*', '~', '_', "'", '"']
replacements = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '', '']

In [ ]:
def preprocessing(reviews, patterns, replacements):
  lst = []
  for i in range(len(reviews)):
    review = reviews[i].lower()
    for pattern, replacement in zip(patterns, replacements):
      review = review.replace(pattern, replacement)
    lst.append(review)
  return lst

In [ ]:
reviews = preprocessing(reviews, patterns, replacements)

In [ ]:
num_train = 35000
num_val = 15000
longest_num_tokens = 500

In [ ]:
def indexing_tokens():
  indices = {'<SOS>':0, '<EOS>': 1, '<PAD>': 2, '<UNK>': 3}
  counter = 4
  for i in range(num_train):
    tokens = reviews[i].split()
    for token in tokens:
      if token not in indices:
        indices[token] = counter
        counter += 1
  return indices


In [ ]:
def get_data(indices, longest_line_tokens, mode='train'):
    data = []
    Y = []
    if mode == 'train':
      for i in range(num_train):
        one_train_data = []
        y, tokens = labels[i], reviews[i].split()
        for token in tokens:
          one_train_data.append(indices[token])
          if len(one_train_data) == longest_line_tokens:
            break
        while len(one_train_data) < longest_line_tokens:
          one_train_data.append(indices['<PAD>'])
        one_train_data.insert(indices['<SOS>'], 0)
        one_train_data.append(indices['<EOS>'])
        data.append(one_train_data)
        Y.append(y)
    else:
      for i in range(num_train, num_train+num_val):
        one_val_data = []
        y, tokens = labels[i], reviews[i].split()
        for token in tokens:
          if token not in indices:
            one_val_data.append(indices['<UNK>'])
          else:
            one_val_data.append(indices[token])
          if len(one_val_data) == longest_line_tokens:
            break
        while len(one_val_data) < longest_line_tokens:
          one_val_data.append(indices['<PAD>'])
        one_val_data.insert(indices['<SOS>'], 0)
        one_val_data.append(indices['<EOS>'])
        data.append(one_val_data)
        Y.append(y)
    return data, Y

In [ ]:
# Loading Training Data & Val Data
indices = indexing_tokens()
training_data, training_labels = get_data(indices, longest_num_tokens)
val_data, val_labels = get_data(indices, longest_num_tokens, mode='val')

In [ ]:
print(training_data[0])

[0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 6, 32, 33, 11, 34, 31, 35, 16, 36, 37, 38, 39, 40, 41, 5, 42, 43, 44, 45, 23, 46, 6, 47, 48, 49, 31, 25, 26, 50, 51, 52, 53, 6, 54, 55, 56, 57, 25, 52, 58, 59, 60, 30, 61, 62, 63, 64, 56, 42, 37, 26, 65, 45, 6, 66, 67, 5, 6, 47, 68, 26, 69, 16, 24, 11, 26, 6, 70, 71, 62, 6, 72, 73, 74, 75, 76, 68, 77, 78, 79, 80, 81, 82, 83, 84, 5, 6, 85, 86, 87, 6, 88, 89, 90, 91, 39, 92, 93, 94, 95, 26, 50, 96, 79, 6, 97, 98, 81, 26, 99, 62, 100, 101, 102, 103, 104, 105, 106, 107, 39, 108, 94, 109, 110, 111, 112, 113, 39, 114, 115, 22, 116, 117, 118, 119, 120, 121, 6, 122, 123, 5, 6, 52, 26, 124, 62, 6, 125, 11, 68, 126, 86, 7, 127, 128, 129, 130, 131, 132, 133, 53, 134, 135, 130, 136, 130, 137, 16, 138, 139, 140, 6, 32, 17, 119, 141, 142, 34, 31, 24, 94, 143, 68, 36, 144, 119, 145, 121, 119, 36, 146, 53, 68, 147, 24, 119, 148, 108, 119, 149, 51, 150, 53, 16, 39, 151, 152, 62, 6, 96, 153, 5, 

In [ ]:
print('Number of training:', len(training_data))
print('Number of validation:', len(val_data))
print('Length of corpus:', len(indices))

Number of training: 35000
Number of validation: 15000
Length of corpus: 122545


In [ ]:
# Create tensors of train & val
train_tensor = torch.tensor(training_data)
train_labels_tensor = torch.tensor(training_labels)
val_tensor = torch.tensor(val_data)
val_labels_tensor = torch.tensor(training_labels)

In [ ]:
print('Train Tensor:', train_tensor.shape)
print('Val Tensor:', val_tensor.shape)

Train Tensor: torch.Size([35000, 502])
Val Tensor: torch.Size([15000, 502])


In [ ]:
vocab_size = 122545
embedding_dim = 300
sequence_len = longest_num_tokens + 2
output_dim = 2
print_every = 200
batch_size = 64
qkv_dim = 200
heads = 8

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, sequence_len, embedding_dim):
    super().__init__()
    self.sequence_len = sequence_len
    self.embedding_dim = embedding_dim
  def forward(self):
    every_other = torch.arange(0, self.embedding_dim, 2).to(device)
    # 150
    denominator = torch.pow(10000, every_other/self.embedding_dim)
    # 150
    positions = torch.arange(self.sequence_len).reshape(self.sequence_len, 1).to(device)
    # (502, 1)
    even = torch.sin(positions/denominator)
    # (502, 150)
    odd = torch.cos(positions/denominator)
    # (502, 150)
    stacked = torch.stack((even, odd), dim=2)
    # (502, 150, 2)
    return torch.flatten(stacked, 1)
    # (502, 300)



class InputEncoding(nn.Module):
  def __init__(self, sequence_len, vocab_size, embedding_dim):
    super().__init__()
    self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
    self.positional_encoding = PositionalEncoding(sequence_len, embedding_dim)
  def forward(self, x):
    emb1 = self.word_embedding(x)
    emb2 = self.positional_encoding()
    return emb1 + emb2

In [ ]:
class FeedForwardLayer(nn.Module):
  def __init__(self, emb_size, d_out):
    super().__init__()
    self.linear1 = nn.Linear(emb_size, d_out)
    self.linear2 = nn.Linear(d_out, emb_size)

  def forward(self, x):
    return self.linear2(nn.functional.dropout(nn.functional.relu(self.linear1(x))))

In [ ]:
class MultiHeadSelfAttention(nn.Module):
  def __init__(self, embedding_dim, qkv_dim, heads):
    super().__init__()
    self.to_q = nn.Linear(embedding_dim, qkv_dim)
    self.to_k = nn.Linear(embedding_dim, qkv_dim)
    self.to_v = nn.Linear(embedding_dim, qkv_dim)
    self.to_out = nn.Linear(qkv_dim, embedding_dim)

  def forward(self, q, k, v):
    N, sequence_len, embeeding_dim = q.shape
    query, key, value = self.to_q(q), self.to_k(k), self.to_v(v) # NQF
    similarity = torch.einsum('NQF,NKF->NQK', [query, key])
    scale = embedding_dim**0.5
    out = torch.softmax(similarity/scale, dim=2)
    out = torch.einsum('NQK,NQF->NKF', [out, value])
    out = self.to_out(out)
    return out

In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self):
    super().__init__()
    self.norm = nn.LayerNorm(embedding_dim)
  def forward(self, x, sub_layer):
    x = sub_layer(x)
    # print(f'after {sub_layer}: {x.shape}')
    x = self.norm(x)
    return x + nn.functional.dropout(x)

In [ ]:
class Encoder(nn.Module):
  def __init__(self, attention: MultiHeadSelfAttention, feed_forward: FeedForwardLayer, N, seq_len, vocab_size, emb_size):
    super().__init__()
    self.encoding = InputEncoding(sequence_len, vocab_size, embedding_dim)
    self.attention = attention
    self.feed_forward = feed_forward
    self.residual = ResidualBlock()
    self.norm = nn.LayerNorm(embedding_dim)
    self.out = nn.Linear(embedding_dim*sequence_len, 2)
  def forward(self, x):
    x = self.encoding(x)
    x = self.residual(x, lambda x: self.attention(x, x, x))
    # The lambda tells our self.residule to take def forward(self, x) instead of the output of forward
    x = self.residual(x, self.feed_forward)
    x = self.norm(x)
    x = torch.flatten(x, 1)
    return self.out(x)

In [ ]:
model = Encoder(MultiHeadSelfAttention(embedding_dim, qkv_dim, heads), FeedForwardLayer(embedding_dim, embedding_dim), batch_size, sequence_len, vocab_size, embedding_dim)
model = model.cuda()

In [ ]:
mini_trains = DataLoader(train_tensor, batch_size=batch_size)
mini_train_labels = DataLoader(training_labels, batch_size=batch_size)

mini_vals = DataLoader(val_tensor, batch_size=batch_size)
mini_val_labels = DataLoader(val_labels, batch_size=batch_size)

In [ ]:
iterator = iter(mini_trains)
print(next(iterator).shape)

iterator = iter(mini_train_labels)
print(next(iterator).shape)

torch.Size([64, 502])
torch.Size([64])


In [ ]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_train_labels, mini_vals, mini_val_labels, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in zip(mini_trains, mini_train_labels):
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % print_every == 0:
        evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device)
      num_iters += 1

In [ ]:
# Evaluate Procedure
def evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in zip(mini_vals, mini_val_labels):
      x=x.to(device)
      y=y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/len(val_data)}')

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Start training
train(4, model, mini_trains, mini_train_labels, mini_vals, mini_val_labels, device, loss_function, optimizer)

Epoch[1] Acc: 0.49933333333333335
Epoch[1] Acc: 0.49766666666666665
Epoch[1] Acc: 0.4968666666666667
Epoch[2] Acc: 0.5040666666666667
Epoch[2] Acc: 0.5854
Epoch[2] Acc: 0.7829333333333334
Epoch[3] Acc: 0.8162
Epoch[3] Acc: 0.8292
Epoch[3] Acc: 0.8614666666666667
Epoch[4] Acc: 0.8513333333333334
Epoch[4] Acc: 0.8708666666666667
Epoch[4] Acc: 0.8329333333333333
